In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from cifar10_models import mobilenetv2
from torchvision import transforms, datasets
from torch.utils.data import Subset, DataLoader, Dataset
import matplotlib.pyplot as plt

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

In [ ]:

Net = mobilenetv2.mobilenet_v2(pretrained=True)

In [ ]:
#for module in Net.modules():
#    if isinstance(module, nn.BatchNorm2d):
#        module.eval()

In [ ]:
#for param in Net.parameters():
#    param.requires_grad = False


In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

In [ ]:
torch.cuda.memory_allocated()/1024**2, torch.cuda.memory_cached()/1024**2

In [ ]:
Net.classifier

In [ ]:
class MyDataset(Dataset):
    def __init__(self, train_x, train_y):
        self.data = train_x 
        self.target = train_y 
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y
    
    def __len__(self):
        return len(self.data)


In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, pretrainedModel):
        super().__init__()
        self.features = pretrainedModel.features
        
        self.Flatten = nn.Flatten()
        self.FC1 = nn.Linear(20480, 32)
        #self.FC2 = nn.Linear(128, 64)
        #self.FC3 = nn.Linear(64, 32)
        #self.FC4 = nn.Linear(32, 32)


        self.Dropout2D = nn.Dropout2d(p=0.2)
        self.Dropout = nn.Dropout(p=0.2)


        self.outLayer1 = nn.Linear(32, 4)
        self.outLayer2 = nn.Linear(32, 4)
        self.outLayer3 = nn.Linear(32, 4)
        self.outLayer4 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.Dropout2D(self.features(x))
        x = self.Flatten(x) # flatten out the last conv layer

        x = self.Dropout(torch.selu(self.FC1(x)))# use dropout with p=0.2
        #x = self.Dropout(torch.selu(self.FC2(x)))
        #x = self.Dropout(torch.selu(self.FC3(x)))
        #x = self.Dropout(torch.selu(self.FC4(x)))

        out1 = self.outLayer1(x)
        out2 = self.outLayer2(x)
        out3 = self.outLayer3(x)
        out4 = self.outLayer4(x)

        return out1, out2, out3, out4


In [ ]:
dataset = datasets.CIFAR10('../data-unversioned/p1ch7/', train=True, transform=preprocess, download=False)
imgs = os.listdir('data/training')
imgs.sort()
indices = [int(name[0:5]) for name in imgs]
my_subset = Subset(dataset, indices) #create subset based on indices


In [ ]:
train_x = []
for data, label in my_subset:
    train_x.append(data)
train_x = torch.stack(train_x)


In [ ]:
labels = torch.from_numpy(np.load('data/labels.npy')).float()
b = labels == 0
indices = b.nonzero()[..., 0]
index = torch.ones(labels.shape[0], dtype=bool)
index[indices] = False

labels = labels[index]
labels = labels.to(device=device)
train_x = train_x[index]

In [ ]:

TrainSet = MyDataset(train_x, labels)

In [ ]:
train_loader = DataLoader(TrainSet, batch_size=64, shuffle=True) 
model = RegressionModel(Net).to(device=device)


In [ ]:
model.training

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
def IoULoss(predictions, ground_truth):
    P_x = predictions[..., 0:1]
    P_y = predictions[..., 1:2]
    P_h = predictions[..., 2:3]
    P_w = predictions[..., 3:4]
    G_x = ground_truth[..., 0:1]
    G_y = ground_truth[..., 1:2]
    G_h = ground_truth[..., 2:3]
    G_w = ground_truth[..., 3:4]

    w_intersection = torch.min(P_x + P_w, G_x + G_w) - torch.max(P_x, G_x)
    h_intersection = torch.min(P_y + P_h, G_y + G_h) - torch.max(P_y, G_y)
    intersection = w_intersection.clamp(0) * h_intersection.clamp(0)#if no intersection, value will default to 0
   
    union = P_h * P_w + G_h * G_w - intersection
    IoU = (intersection + 1e-6)/(union + 1e-6)

    ##central points
    central_p_x = (P_x + P_w)/2
    central_p_y = (P_y + P_h)/2
    central_l_x = (G_x + G_w)/2
    central_l_y = (G_y + G_h)/2
    euc_dist = torch.sqrt(torch.square(central_l_x - central_p_x) + torch.square(central_l_y - central_p_y))
    
    
    
    #get diagional
    w_union = torch.max(P_x + P_w, G_x + G_w) - torch.min(P_x, G_x)
    h_union = torch.max(P_y + P_h, G_y + G_h) - torch.min(P_y, G_y)
    c_diag = torch.sqrt(torch.square(w_union) + torch.square(h_union))

    #penalty term
    penalty_term = euc_dist/c_diag

    DistanceIoULoss = 1 - IoU + penalty_term

    return DistanceIoULoss.mean()

def CIoULoss(predictions, ground_truth):
    P_x = predictions[..., 0:1]
    P_y = predictions[..., 1:2]
    P_h = predictions[..., 2:3]
    P_w = predictions[..., 3:4]
    G_x = ground_truth[..., 0:1]
    G_y = ground_truth[..., 1:2]
    G_h = ground_truth[..., 2:3]
    G_w = ground_truth[..., 3:4]

    w_intersection = torch.min(P_x + P_w, G_x + G_w) - torch.max(P_x, G_x)
    h_intersection = torch.min(P_y + P_h, G_y + G_h) - torch.max(P_y, G_y)
    intersection = w_intersection.clamp(0) * h_intersection.clamp(0)#if no intersection, value will default to 0
   
    union = P_h * P_w + G_h * G_w - intersection
    IoU = (intersection + 1e-6)/(union + 1e-6)

    ##central points
    central_p_x = (P_x + P_w)/2
    central_p_y = (P_y + P_h)/2
    central_l_x = (G_x + G_w)/2
    central_l_y = (G_y + G_h)/2
    euc_dist = torch.sqrt(torch.square(central_l_x - central_p_x) + torch.square(central_l_y - central_p_y))
    
    
    
    #get diagional
    w_union = torch.max(P_x + P_w, G_x + G_w) - torch.min(P_x, G_x)
    h_union = torch.max(P_y + P_h, G_y + G_h) - torch.min(P_y, G_y)
    c_diag = torch.sqrt(torch.square(w_union) + torch.square(h_union))

    #penalty term
    penalty_term = euc_dist/c_diag


    #aspect ratio
    pi = torch.acos(torch.zeros(1)).item() * 2
    v = 4/(pi**2) * torch.square(torch.atan(G_w/G_h) - torch.atan(P_w/P_h)) #aspect ratio
    alpha =  v/((1 - IoU) + v)
 

    CompleteIoULoss = 1 - IoU + penalty_term + (alpha * v)

    return CompleteIoULoss.mean()

In [ ]:
loss_d = IoULoss
loss_c = CIoULoss
loss_quad = nn.MSELoss()
loss_abs = nn.L1Loss()

In [ ]:
def arrange_labels_correctly(label_batch):
    l1 = torch.stack([label[0] for label in label_batch])
    l2 = torch.stack([label[1] for label in label_batch])
    l3 = torch.stack([label[2] for label in label_batch])
    l4 = torch.stack([label[3] for label in label_batch])

    return l1, l2, l3, l4

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_IoU, loss_abs, loader, batch_size: int):    
    losses = []

    for epoch in range(1, n_epochs+1):
        running_loss = 0.0
        idx = 0
        for img_batch, labels in loader:
            img_batch = img_batch.to(device=device)
            out1, out2, out3, out4 = model(img_batch)
            #out1 = model(img_batch)
            label1, label2, label3, label4 = arrange_labels_correctly(labels)
            
            loss1 = loss_IoU(out1, label1) + loss_abs(out1, label1)
            loss2 = loss_IoU(out2, label2) + loss_abs(out2, label2) 
            loss3 = loss_IoU(out3, label3) + loss_abs(out3, label3)
            loss4 = loss_IoU(out4, label4) + loss_abs(out4, label4)

            #loss_total = loss1
            #print(loss_total)
            loss_total = (loss1 + loss2 + loss3 + loss4)#accumulate loss 
            optimizer.zero_grad()

            loss_total.backward()
            optimizer.step()
            running_loss += loss_total.item() * img_batch.size(0)
            
            if idx % 400 == 0:
                print(f'step {idx} is the current iteration and loss is: {loss_total}')

            idx += 1
            
        
        epoch_loss = running_loss / len(TrainSet)
        print(f'At epoch: {epoch}, the training loss is {epoch_loss}')
        losses.append(epoch_loss)
        

In [ ]:
training_loop(n_epochs=21, optimizer=optimizer, model=model, loss_IoU=loss_d, loss_abs=loss_abs, loader=train_loader, batch_size=64)

In [ ]:
training_loop(n_epochs=12, optimizer=optimizer, model=model, loss_fn=loss_quad, loader=train_loader, batch_size=64)


In [ ]:
training_loop(n_epochs=20, optimizer=optimizer, model=model, loss_fn=loss_c, loader=train_loader, batch_size=64)

In [ ]:
training_loop(n_epochs=30, optimizer=optimizer, model=model, loss_fn=loss, loader=train_loader, batch_size=64)

In [ ]:
model.train()
model.training

In [ ]:
for imgbatch, labels in train_loader:
    o1 = model(imgbatch.to(device=device))
    
    asd = labels
    break

In [ ]:
loss(o1, l1)

In [ ]:
asd.shape

In [ ]:
l1 = torch.stack([a[0][:2] for a in asd])
l1.shape

In [ ]:

labels[0], l1[0]

In [ ]:
del model
torch.cuda.empty_cache()
del labels
torch.cuda.empty_cache()

In [ ]:
model.eval()

print("hello")

In [ ]:
model.training

In [ ]:
asd2 = my_subset[974][0].to(device=device)
asd3 = my_subset[1852][0].to(device=device)

In [ ]:
model(torch.unsqueeze(asd2, 0)), model(torch.unsqueeze(asd3, 0))

In [ ]:
labels[974][0], labels[1852][0]

In [ ]:
img_p = asd2
rect = patches.Rectangle((14.2388, 15.0060), 2.7476, 2.3851, linewidth=1, edgecolor='k', facecolor='k')
#rect2 = patches.Rectangle((14.1871, 15.7513), 4.1633, 4.0735, linewidth=1, edgecolor='k', facecolor='k')
#rect3 = patches.Rectangle((15.5461,  14.7842), 3.6610, 4.1208, linewidth=1, edgecolor='w', facecolor='w')
#rect4 = patches.Rectangle((15.3920,  15.0777), 4.6964, 4.7229, linewidth=1, edgecolor='w', facecolor='w')

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(img_p)
ax.add_patch(rect)
ax.add_patch(rect2)
ax.add_patch(rect3)
ax.add_patch(rect4)
plt.show()